In [1]:
import os
from getpass import getpass
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("openai_key") or \
  getpass("Enter your OpenAI API key: ")

In [2]:
from agents import Agent, Runner
from openai.types.responses import ResponseTextDeltaEvent

agent = Agent(
    name="Assistant",
    instructions="You're a helpful assistant",
    model="gpt-4o",
)

OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export
OPENAI_API_KEY is not set, skipping trace export


We can filter these various event types to find only raw tokens like so:

## Tools

In [3]:
from agents import function_tool
from typing import List, Union, Optional, Tuple

# @function_tool
# def multiply(x: float, y: float) -> float:
#     """Multiplies `x` and `y` to provide a precise
#     answer."""
#     return x*y


from typing import List, Optional, Union, Tuple

from typing import List, Optional, Union, Tuple, Dict

@function_tool
def build_query(
    select_columns: List[str],
    table_name: str,
    where_conditions: Optional[Union[
        Dict[str, Union[str, int, float]],
        List[List[Union[str, int, float]]]  # Changed from Tuple[...] to List[...]
    ]] = None,
    group_by_columns: Optional[List[str]] = None,
    having_conditions: Optional[Union[
        Dict[str, Union[str, int, float]],
        List[List[Union[str, int, float]]]  # Changed from Tuple[...] to List[...]
    ]] = None
) -> str:
    query = f"SELECT {', '.join(select_columns)} FROM {table_name}"
    
    # Process WHERE conditions
    if where_conditions:
        if isinstance(where_conditions, dict):
            conditions = " AND ".join(
                f"{col} = '{val}'" if isinstance(val, str) else f"{col} = {val}"
                for col, val in where_conditions.items()
            )
        else:  # List of tuples
            conditions = " AND ".join(
                f"{col} {op} '{val}'" if isinstance(val, str) else f"{col} {op} {val}"
                for col, op, val in where_conditions
            )
        query += f" WHERE {conditions}"
    
    # Process GROUP BY and HAVING conditions
    if group_by_columns:
        query += f" GROUP BY {', '.join(group_by_columns)}"
        if having_conditions:
            if isinstance(having_conditions, dict):
                conditions = " AND ".join(
                    f"{col} = '{val}'" if isinstance(val, str) else f"{col} = {val}"
                    for col, val in having_conditions.items()
                )
            else:
                conditions = " AND ".join(
                    f"{col} {op} '{val}'" if isinstance(val, str) else f"{col} {op} {val}"
                    for col, op, val in having_conditions
                )
            query += f" HAVING {conditions}"
    
    return query


In [4]:
import pymysql
import urllib.parse  # To handle special characters in password
from sqlalchemy import create_engine

@function_tool
def execute_sql_query(sql_query: str) -> List:
    # print(f"--- Received request to execute SQL: {sql_query[:100]}... ---") # Log snippet

    if not sql_query.strip().upper().startswith("SELECT"):
        error_msg = "SECURITY ERROR: Only SELECT queries are permitted."
        print(f"--- {error_msg} ---")

        return [{"error": error_msg}]

    conn = None
    results = []
    try:
        # ------------------------------------------------------------------
        DB_HOST = "127.0.0.1"  # Localhost
        DB_PORT = 3306  # Default MySQL port
        DB_USER = "root"
        DB_PASS = "12345.Manan#"
        DB_NAME = "semb_prod_new"

        def connect_with_connector():
            """Establish a connection to the local MySQL database."""
            def getconn():
                return pymysql.connect(
                    host=DB_HOST,
                    user=DB_USER,
                    password=DB_PASS,
                    database=DB_NAME,
                    port=DB_PORT
                )
            return create_engine("mysql+pymysql://", creator=getconn)
        engine = connect_with_connector()
        with engine.connect() as conn:
        # --- Execute the Query ---
            data_retrieved = conn.execute(sql_query).fetchall()


            results = [dict(row) for row in data_retrieved]
            return results
    except Exception as e:
        return [{"error": error_msg}]



In [5]:
agent = Agent(
    name="Assistant",
    instructions=(
        "You're an experienced data analyse "
        "use the provided tools whenever possible. Do not "
        "rely on your own knowledge too much and instead "
        "use your tools to help you answer queries.",
        "I AM PROVIDING you with tool to retrive the data using retrieve relevent data IF REQUIRED this function"
        "contains parameters:-1. build_query(select_columns, table_name, where_conditions=None, group_by_columns=None, having_conditions=None)"
        "2. execute_sql_query(sql_query: str) -> List"
        "NOTE THAT I WILL BE PROVIDING YOU THE DETAILS AND USE THIS FUNCTION  AND ADJUST THE PARAMETERS RESPECTIVELY BASED ON MY AS TO RETRIEVE THE DATA AND GIVE ME THE ANSWER"
    ),
    model="gpt-4o",
    tools=[build_query, execute_sql_query]  # note that we expect a list of tools
)

Now let's initialize a new runner and execute our agent with tools:

In [7]:

response = Runner.run_streamed(
    starting_agent=agent,
    input=" In table steps the step_id 22 was supposed to be executed before step_id 32, but the opposite happened this case could only happen when " \
    "custom_input column of step_id 25 is an empty list [], how do i figure this out using steps table how do i view that table also mention if you used the tool to determine or not"
)
        
async for event in response.stream_events():
    if event.type == "raw_response_event" and \
        isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Instructions must be a string or a function, got ("You're an experienced data analyse use the provided tools whenever possible. Do not rely on your own knowledge too much and instead use your tools to help you answer queries.", 'I AM PROVIDING you with tool to retrive the data using retrieve relevent data IF REQUIRED this functioncontains parameters:-1. build_query(select_columns, table_name, where_conditions=None, group_by_columns=None, having_conditions=None)2. execute_sql_query(sql_query: str) -> ListNOTE THAT I WILL BE PROVIDING YOU THE DETAILS AND USE THIS FUNCTION  AND ADJUST THE PARAMETERS RESPECTIVELY BASED ON MY AS TO RETRIEVE THE DATA AND GIVE ME THE ANSWER')


XXXXXXXXXXXX
[FunctionTool(name='build_query', description='', params_json_schema={'properties': {'select_columns': {'items': {'type': 'string'}, 'title': 'Select Columns', 'type': 'array'}, 'table_name': {'title': 'Table Name', 'type': 'string'}, 'where_conditions': {'anyOf': [{'additionalProperties': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'type': 'number'}]}, 'type': 'object'}, {'items': {'items': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'type': 'number'}]}, 'type': 'array'}, 'type': 'array'}, {'type': 'null'}], 'title': 'Where Conditions'}, 'group_by_columns': {'anyOf': [{'items': {'type': 'string'}, 'type': 'array'}, {'type': 'null'}], 'title': 'Group By Columns'}, 'having_conditions': {'anyOf': [{'additionalProperties': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'type': 'number'}]}, 'type': 'object'}, {'items': {'items': {'anyOf': [{'type': 'string'}, {'type': 'integer'}, {'type': 'number'}]}, 'type': 'array'}, 'type': 'array'}, {'type': 'null'}

Error streaming response: Connection error.


APIConnectionError: Connection error.